In [71]:
import os
import numpy as np

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend
from tensorflow import distribute
import pickle
import data_processing as dp

In [72]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Flatten, UpSampling2D

In [73]:
import tensorflow as tf

# Get the list of GPUs available
gpus = tf.config.experimental.list_physical_devices('GPU')
# Enable memory growth on each GPU
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
if gpus:
    try:
        # Set memory limit for the first GPU
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])  # Limit to 1GB, for example
    except RuntimeError as e:
        print(e)

In [74]:
BATCH_SIZE = 32 
learning_rate = 0.005
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
image_size = (32, 32, 3)
sampling = 3

In [75]:
(x_train, y_train), (x_test, y_test) = dp.load_data_keras("../Data")

In [76]:
#(training_images, training_labels) , (validation_images, validation_labels) = tf.keras.datasets.cifar10.load_data()
def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  input_images = UpSampling2D(size=(sampling, sampling))(input_images)
  output_ims = preprocess_input(input_images)
  return output_ims
strategy = distribute.get_strategy()
with strategy.scope():
    fe_model = ResNet50(weights='imagenet', include_top=False, input_shape=(image_size[0] * sampling, image_size[1] * sampling, image_size[2]))



In [77]:
features_train = fe_model.predict(preprocess_image_input(x_train))
features_test = fe_model.predict(preprocess_image_input(x_test))

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 214s 136ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 45s 145ms/step


In [104]:
import matplotlib.pyplot as plt
from keras.preprocessing import image

In [79]:
n_features = features_train.shape[1:]
log_model = Sequential([
    Input(shape=n_features),
    Flatten(),
    Dense(10, activation='softmax')
])
log_model.compile(optimizer=Adam(learning_rate=learning_rate), loss=SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
log_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_2 (Flatten)             │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │       184,330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 184,330 (720.04 KB)

 Trainable params: 184,330 (720.04 KB)

 Non-trainable params: 0 (0.00 B)

In [80]:
checkpoint = ModelCheckpoint(
    'model/resnet_model.keras',             # Path where to save the model
    monitor='val_accuracy',       # Metric to monitor
    save_best_only=True,          # Save only the best model
    mode='max',                   # Mode for choosing the best value ('max' since higher accuracy is better)
    verbose=1                     # Verbose output to see when checkpoints are saved
)
log_model.fit(features_train, y_train, batch_size=BATCH_SIZE, epochs=10, validation_data=(features_test, y_test), callbacks=[checkpoint])

Epoch 1/10
1557/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6726 - loss: 15.1269
Epoch 1: val_accuracy improved from -inf to 0.75460, saving model to model/resnet_model.keras
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6728 - loss: 15.1275 - val_accuracy: 0.7546 - val_loss: 15.8412
Epoch 2/10
1526/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8155 - loss: 10.0925
Epoch 2: val_accuracy improved from 0.75460 to 0.76050, saving model to model/resnet_model.keras
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8155 - loss: 10.1070 - val_accuracy: 0.7605 - val_loss: 18.3255
Epoch 3/10
1551/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8581 - loss: 7.6740
Epoch 3: val_accuracy improved from 0.76050 to 0.78620, saving model to model/resnet_model.keras
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8580 - loss: 7.6813 - val_accuracy: 0.7862 - val_loss: 18.4431
Epoch 4/10
1521/1563 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8815 - loss: 6.5

In [81]:
from sklearn.metrics import classification_report
probabilities = log_model.predict(features_test, batch_size=128)

probabilities = np.argmax(probabilities, axis = 1)

print(classification_report(y_test, probabilities, target_names= ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'], digits=4))

79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 868us/step
              precision    recall  f1-score   support

    airplane     0.8332    0.8140    0.8235      1000
  automobile     0.8825    0.8940    0.8882      1000
        bird     0.7640    0.7090    0.7355      1000
         cat     0.5956    0.6540    0.6235      1000
        deer     0.7548    0.7140    0.7338      1000
         dog     0.6132    0.8260    0.7039      1000
        frog     0.9154    0.6710    0.7744      1000
       horse     0.8486    0.8130    0.8304      1000
        ship     0.9182    0.8760    0.8966      1000
       truck     0.8566    0.8960    0.8759      1000

    accuracy                         0.7867     10000
   macro avg     0.7982    0.7867    0.7886     10000
weighted avg     0.7982    0.7867    0.7886     10000

